In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation, metrics 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.matrix as smatrix
import random
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score, KFold

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def get_label_maps(df_train):
    flatten = lambda l: [item for sublist in l for item in sublist]
    unique_labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

    label_map1 = {lbl: idx for idx, lbl in enumerate(unique_labels)}
    inv_label_map = {idx: lbl for lbl, idx in label_map1.items()}
    return label_map1, inv_label_map

df_train = pd.read_csv('../data/train_v2.csv', index_col='image_name')
label_map1, inv_label_map = get_label_maps(df_train)

In [3]:
df_train = pd.read_csv('../data/train_v2.csv')
labels = df_train['tags'].apply(lambda x: x.split(' '))

weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
weather = []

for i in [l.split(' ') for l in df_train['tags'].values]:
    for j in i:
        if j in weather_labels:
            weather += [j]

df_train['weather'] = weather

In [4]:
categories = np.array(['primary', 'artisinal_mine', 'cloudy','blooming','conventional_mine',
                       'partly_cloudy','road','water','selective_logging','blow_down',
                       'cultivation','agriculture','haze','habitation',
                       'slash_burn','bare_ground','clear'])

label_map = {l: i for i, l in enumerate(categories)}


#labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))
#label_map = {l: i for i, l in enumerate(labels)}
weather_label_map = {l: i for i, l in enumerate(weather_labels)}
  

### мои фолды

In [5]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(4, shuffle=True, random_state=1)
folds = []
for itr, ite in skf.split(df_train, df_train.weather):
    folds += [[itr, ite]]

### фолды asanakoy

In [6]:
from sklearn.model_selection import KFold    
#df_train = pd.read_csv('train_v2.csv', index_col='image_name')

n_folds = 5
as_kf = KFold(n_splits=5, random_state=2002, shuffle=True)
#as_folds = list(kfold.split(df_train))

as_folds = []
for itr, ite in as_kf.split(df_train):
    as_folds += [[itr, ite]]

#FOLD_ID = 0 # from 0 to 4
#assert 0 <= FOLD_ID < n_folds
#as_folds = folds[FOLD_ID:FOLD_ID + 1]
#for train_index, valid_index in folds:
#    ids_train_fold, ids_valid_fold = df_train.index[train_index], df_train.index[valid_index]
	# ids_valid_fold is what you need for each fold

## train

In [7]:
df = []
for i in range(1, 5):
    df += [pd.read_csv('resnet_101/fold'+str(i)+'/train.csv', header=0)]
    df[i-1]['idx'] = folds[i-1][1]
tr1 = pd.concat([df[0], df[1], df[2], df[3]], axis=0)
tr1 = tr1.sort_values('idx')
del tr1['idx']
tr1.index = range(len(tr1))

In [8]:
df = []
for i in range(1, 5):
    df += [pd.read_csv('resnet_152/fold'+str(i)+'/train.csv', header=0)]
    df[i-1]['idx'] = folds[i-1][1]
tr2 = pd.concat([df[0], df[1], df[2], df[3]], axis=0)
tr2 = tr2.sort_values('idx')
del tr2['idx']
tr2.index = range(len(tr2))

In [9]:
df = []
for i in range(1, 5):
    df += [pd.read_csv('VGG16/fold'+str(i)+'/train.csv', header=0)]
    df[i-1]['idx'] = folds[i-1][1]
tr3 = pd.concat([df[0], df[1], df[2], df[3]], axis=0)
tr3 = tr3.sort_values('idx')
del tr3['idx']
tr3.index = range(len(tr3))

In [11]:
for i in range(5):
    print(len(np.load('asanakoy/resnet50/train/fold'+str(i)+'.npy')))

40479
40479
40479
40479
40479


In [10]:
df = []
for i in range(5):
    df += [pd.DataFrame(np.load('asanakoy/resnet50/train/fold'+str(i)+'.npy'))]
#    df[i]['idx'] = as_folds[i][1]

#tr4 = pd.concat([df[0], df[1], df[2], df[3], df[4]], axis=0)

#tr4 = (df[0]+ df[1]+ df[2]+ df[3]+ df[4]) / 5

#tr4 = tr4.sort_values('idx')
#del tr4['idx']
#tr4.index = range(len(tr4))

In [11]:
#tr4 = (df[0]+ df[1]+ df[2]+ df[3]+ df[4]) / 5


In [12]:
tr4 = pd.concat([df[0].ix[as_folds[0][1]],
          df[1].ix[as_folds[1][1]],
          df[2].ix[as_folds[2][1]],
          df[3].ix[as_folds[3][1]],
          df[4].ix[as_folds[4][1]]], axis=0).sort_index()

In [13]:
new_tr4 = tr4 * 0
for i in label_map:
    new_tr4[label_map[i]] = tr4[label_map1[i]]
tr4 = pd.DataFrame(new_tr4)

In [14]:
tr4 = pd.read_csv('asa.csv')


In [15]:
df = []
for i in range(1, 5):
    df += [pd.read_csv('VGG19/fold'+str(i)+'/train.csv', header=0)]
    df[i-1]['idx'] = folds[i-1][1]
tr5 = pd.concat([df[0], df[1], df[2], df[3]], axis=0)
tr5 = tr5.sort_values('idx')
del tr5['idx']
tr5.index = range(len(tr5))

In [16]:
df = []
for i in range(1, 5):
    df += [pd.read_csv('Inception_v3/fold'+str(i)+'/train.csv', header=0)]
    df[i-1]['idx'] = folds[i-1][1]
tr6 = pd.concat([df[0], df[1], df[2], df[3]], axis=0)
tr6 = tr6.sort_values('idx')
del tr6['idx']
tr6.index = range(len(tr6))

In [17]:
df = []
for i in range(1, 5):
    df += [pd.read_csv('weather/fold'+str(i)+'/train.csv', header=0)]
    df[i-1]['idx'] = folds[i-1][1]
tr7 = pd.concat([df[0], df[1], df[2], df[3]], axis=0)
tr7 = tr7.sort_values('idx')
del tr7['idx']
tr7.index = range(len(tr7))

In [19]:
df = []
for i in range(1, 5):
    df += [pd.read_csv('VGG19_fail/fold'+str(i)+'/train.csv', header=0)]
    df[i-1]['idx'] = folds[i-1][1]
tr8 = pd.concat([df[0], df[1], df[2], df[3]], axis=0)
tr8 = tr8.sort_values('idx')
del tr8['idx']
tr8.index = range(len(tr8))

In [20]:
tr = pd.DataFrame((((tr1+tr2+tr3+tr5+tr6+tr8 ) / 6).values * 0.6+ tr4.values * 0.4))

In [312]:
tr = pd.DataFrame(((tr1+tr2+tr3+tr5+tr6 ) / 5).values)

## test

In [21]:
te1 = []
for i in range(1, 5):
    te1+= [pd.read_csv("resnet_101/fold"+str(i)+"/test.csv", header=0)]
ave1 = te1[0]*0
for i in range(4):
    ave1 = ave1 + te1[i]
ave1 = ave1 / 4


In [22]:
te2 = []
for i in range(1, 5):
    te2 += [pd.read_csv("resnet_152/fold"+str(i) + "/test.csv", header=0)]
ave2 = te2[0]*0
for i in range(4):
    ave2 = ave2 + te2[i]
ave2 = ave2 / 4



In [23]:
te3 = []
for i in range(1, 5):
    te3 += [pd.read_csv("VGG16/fold"+str(i) + "/test.csv", header=0)]
ave3 = te3[0]*0
for i in range(4):
    ave3 = ave3 + te3[i]
ave3 = ave3 / 4

In [24]:
te4 = []
for i in range(5):
    te4 += [pd.DataFrame(np.load("asanakoy/resnet50/fold"+str(i)+".npy"))]
    tmp = te4[i] * 0
    for k in label_map:
        tmp[label_map[k]] = te4[i][label_map1[k]] 
    te4[i] = tmp
ave4 = te4[0]*0
for i in range(5):
    ave4 = ave4 + te4[i]
ave4 = ave4 / 5

In [25]:
ave4 = pd.read_csv('asa_test.csv')


In [26]:
te5 = []
for i in range(1, 5):
    te5 += [pd.read_csv("VGG19/fold"+str(i) + "/test.csv", header=0)]
ave5 = te5[0]*0
for i in range(4):
    ave5 = ave5 + te5[i]
ave5 = ave5 / 4

In [27]:
te6 = []
for i in range(1, 5):
    te6 += [pd.read_csv("Inception_v3/fold"+str(i) + "/test.csv", header=0)]
ave6 = te6[0]*0
for i in range(4):
    ave6 = ave6 + te6[i]
ave6 = ave6 / 4

In [29]:
te7 = []
for i in range(1, 5):
    te7 += [pd.read_csv("weather/fold"+str(i) + "/test.csv", header=0)]
ave7 = te7[0]*0
for i in range(4):
    ave7 = ave7 + te7[i]
ave7 = ave7 / 4

In [30]:
te8 = []
for i in range(1, 5):
    te8 += [pd.read_csv("VGG19_fail/fold"+str(i) + "/test.csv", header=0)]
ave8 = te8[0]*0
for i in range(4):
    ave8 = ave8 + te8[i]
ave8 = ave8 / 4

In [168]:
ave5 = pd.DataFrame(np.load("asanakoy/inception_v3/test.npy"))
tmp = ave5 * 0
for k in label_map:
    tmp[label_map[k]] = ave5[label_map1[k]] 
ave5 = tmp

In [170]:
ave6 = pd.DataFrame(np.load("asanakoy/VGG19/test.npy"))
tmp = ave6 * 0
for k in label_map:
    tmp[label_map[k]] = ave6[label_map1[k]] 
ave6 = tmp

In [31]:
ave = (ave1+ave2+ave3+ave5+ave6) / 5

In [34]:
blend = pd.DataFrame(ave4.values*0.4 + ave.values*0.6)

## трейн метки

In [31]:
y_all = []
for f, tags, weather in df_train.values:
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    y_all += [targets]
y_all = pd.DataFrame(y_all)

## Скоры

### дефолтные

In [32]:
from sklearn.metrics import fbeta_score

print(fbeta_score(y_all, np.array(tr1) > 0.2, beta=2, average='samples'))
print(fbeta_score(y_all, np.array(tr2) > 0.2, beta=2, average='samples'))
print(fbeta_score(y_all, np.array(tr3) > 0.2, beta=2, average='samples'))
print(fbeta_score(y_all, np.array(tr4) > 0.2, beta=2, average='samples'))
print(fbeta_score(y_all, np.array(tr5) > 0.2, beta=2, average='samples'))
print(fbeta_score(y_all, np.array(tr6) > 0.2, beta=2, average='samples'))

print(fbeta_score(y_all, np.array(tr) > 0.2, beta=2, average='samples'))

0.923940216783
0.921190038809
0.920554321167
0.926291514796
0.923354667401
0.922885248901
0.930319674782


In [ ]:
0.923940216783
0.921190038809
0.920554321167
0.926291514796
0.923354667401
0.922885248901
0.930597913205

In [ ]:
0.930680003768
0.930597913205

In [ ]:
0.930194455376

In [ ]:
0.929881697222

In [ ]:
0.928567876838

In [ ]:
0.927406709985

In [ ]:
0.923940216783
0.921190038809
0.920554321167
0.926291514796
0.929469718062

### с подбором трешхолдов

In [34]:
tr_all = [tr1, tr2, tr3, tr4, tr5, tr6,tr8, tr]

In [35]:
q_all = []
for j in range(7,8):
    ma = 0
    q=[0.2]*17
    for t in range(17):
        ib = 0.2
        for i in np.linspace(0.05, 0.3, 50):
            q[t] = i
            cur = fbeta_score(y_all, np.greater(np.array(tr_all[j]), q), beta=2, average='samples')
            if (cur > ma):
                ma = cur
                ib = i
        q[t] = ib
    q_all += [q]
    print(ma)

0.931434613195


In [ ]:
0.931614474571

In [ ]:
0.931298423122

In [ ]:
0.925165066646
0.922932640406
0.921948863122
0.927589436182
0.930677501976

In [391]:
#q_all = []
for j in range(5):
    ma = fbeta_score(y_all, np.greater(np.array(tr_all[j]), q_all[j]), beta=2, average='samples')
    #q=[0.2]*17
    q = q_all[j]
    for t in range(17):
        ib = q_all[j][t]
        for i in np.linspace(0.05, 0.3, 50):
            q[t] = i
            cur = fbeta_score(y_all, np.greater(np.array(tr_all[j]), q), beta=2, average='samples')
            if (cur > ma):
                ma = cur
                ib = i
        q[t] = ib
    q_all[j] = q
    print(ma)

0.925165066646
0.922933610579
0.921948863122
0.927589436182
0.930677501976


In [142]:
q_all[4]

[0.28469387755102038,
 0.14693877551020407,
 0.12142857142857143,
 0.23877551020408161,
 0.1877551020408163,
 0.2,
 0.22346938775510206,
 0.21836734693877552,
 0.11122448979591837,
 0.11632653061224489,
 0.19795918367346937,
 0.20816326530612245,
 0.22346938775510206,
 0.15714285714285714,
 0.17244897959183675,
 0.1520408163265306,
 0.25918367346938775]

## Валидация

In [37]:
from sklearn.model_selection import KFold    
#df_train = pd.read_csv('train_v2.csv', index_col='image_name')

kf = KFold(n_splits=5, random_state=10, shuffle=True)
cv_folds = []
for itr, ite in kf.split(df_train):
    cv_folds += [[itr, ite]]

## модель второго уровня

In [38]:
tr_conc = pd.concat([tr1, tr2, tr3, tr5, tr6, tr7, tr8], axis=1)
te_conc = pd.concat([ave1, ave2, ave3, ave5, ave6, ave7, ave8], axis=1)

In [59]:
import lightgbm as lgb


In [60]:
pred_train = np.zeros((len(y_all), 17))
sc,sc_mean = [],[]

params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.05,
    'min_child_weight': 1,
    #'max_depth': 7
    #'num_class': 3
    
}
from sklearn.ensemble import ExtraTreesClassifier

for itr, ite in cv_folds:
    from xgboost import XGBClassifier
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.multioutput import MultiOutputClassifier
    
    # If you want to avoid the OneVsRestClassifier magic switch
    # from sklearn.multioutput import MultiOutputClassifier
    #clf_multilabel = OneVsRestClassifier(XGBClassifier(max_depth=2,learning_rate=0.1,n_estimators=100,min_child_weight=1))
    #clf_multilabel = OneVsRestClassifier(LogisticRegression())
    #clf_multilabel = OneVsRestClassifier(ExtraTreesClassifier(n_estimators=500, criterion='gini', max_depth=None))
    clf_multilabel = OneVsRestClassifier(lgb.sklearn.LGBMClassifier(n_estimators=150, seed=0, **params))    
    clf_multilabel.fit(tr_conc.ix[itr].values, y_all.ix[itr].values)
    pred_train[ite] = clf_multilabel.predict_proba(tr_conc.ix[ite].values)
    loc_sc = fbeta_score(y_all.ix[ite, :], pred_train[ite] > 0.2, beta=2, average='samples')
    print(loc_sc)
    sc.append(loc_sc)
print('XGB: {:.7f} +- {:.7f}'.format(np.mean(sc), np.std(sc)))

0.931281555493
0.927431859689
0.92974221138
0.929242122832
0.928508875515
XGB: 0.9292413 +- 0.0012825


In [ ]:
lgb
0.931281555493
0.927431859689
0.92974221138
0.929242122832
0.928508875515
XGB: 0.9292413 +- 0.0012825

In [ ]:
ext
0.931531284908
0.928435852382
0.929534133088
0.92912937538
0.928978721867
XGB: 0.9295219 +- 0.0010646

In [ ]:
все кроме аса
0.932518813701
0.929238708513
0.930865753919
0.930662686451
0.930655202032
XGB: 0.9307882 +- 0.0010426

In [ ]:
0.933309758046
0.928583782536
0.930267854954
0.930231368773
0.93039906491
XGB: 0.9305584 +- 0.0015288

In [ ]:
0.933309758046

In [ ]:
0.932839340317

In [ ]:
0.933822510095
0.929195652376
0.930254248762
0.930882339822
0.931126247263
XGB: 0.9310562 +- 0.0015357

In [ ]:
0.932848301499
0.928869141334
0.93039294455
0.929984599408
0.930869395107
XGB: 0.9305929 +- 0.0013073

In [ ]:
0.932737092693
0.929260698142
0.928336012382
0.929768274658
0.930121586382
XGB: 0.9300447 +- 0.0014741

In [46]:
ma = 0
q=[0.2]*17
for t in range(17):
    ib = 0.2
    for i in np.linspace(0.05, 0.3, 50):
        q[t] = i
        cur = fbeta_score(y_all, np.greater(np.array(pred_train), q), beta=2, average='samples')
        if (cur > ma):
            ma = cur
            ib = i
    q[t] = ib
print(ma)

0.929303430684


In [ ]:
0.931599919004

In [ ]:
0.931798389345

In [121]:
q_stack = q

In [126]:
q_stack

[0.21836734693877552,
 0.20306122448979591,
 0.095918367346938774,
 0.27448979591836731,
 0.26428571428571429,
 0.12653061224489795,
 0.19795918367346937,
 0.21836734693877552,
 0.23367346938775507,
 0.22346938775510206,
 0.21836734693877552,
 0.20816326530612245,
 0.18265306122448977,
 0.19795918367346937,
 0.20306122448979591,
 0.16734693877551021,
 0.16224489795918368]

In [ ]:
0.931798389345

In [ ]:
0.93140019935

In [ ]:
0.930719642795

In [38]:
fbeta_score(y_all, np.greater(np.array(pred_train), q), beta=2, average='samples')

0.92876002648202127

## второй уровень

In [43]:
bst_pred = pred_train

In [47]:
lin_pred = pred_train

In [54]:
ext_pred = pred_train

In [61]:
lgb_pred = pred_train

In [102]:
nn_pred = pred_train

In [137]:
nn_pred2 = pred_train

In [138]:
pred_train = (lin_pred*0.14 + bst_pred*0.3+ ext_pred*0.14+lgb_pred*0.14+nn_pred*0.14+nn_pred2*0.14)

In [136]:
0.7 / 5

0.13999999999999999

In [139]:
ma = 0
q=[0.2]*17
for t in range(17):
    ib = 0.2
    for i in np.linspace(0.05, 0.3, 100):
        q[t] = i
        cur = fbeta_score(y_all, np.greater(np.array(pred_train), q), beta=2, average='samples')
        if (cur > ma):
            ma = cur
            ib = i
    q[t] = ib
print(ma)

0.93219020205


In [ ]:
0.93219020205

In [ ]:
0.932143915682

In [ ]:
0.932066260632

In [ ]:
0.932029635321
0.9319594244

## стекинг + asanakoy

In [140]:
tr = pd.DataFrame((pred_train * 0.65+ tr4.values * 0.35))
#tr = pd.DataFrame(pred_train)

In [141]:
ma = 0
q=[0.2]*17
for t in range(17):
    ib = 0.2
    for i in np.linspace(0.01, 0.4, 150):
        q[t] = i
        cur = fbeta_score(y_all, np.greater(np.array(tr), q), beta=2, average='samples')
        if (cur > ma):
            ma = cur
            ib = i
    q[t] = ib
print(ma)

0.932521112002


In [ ]:
0.932521112002

In [ ]:
0.932390846032

In [ ]:
0.932459845813

In [ ]:
0.932396806077

In [ ]:
0.932263096449

In [ ]:
0.932188931342

In [ ]:
0.932026601785

In [ ]:
0.931845707729

In [113]:
q

[0.27174496644295304,
 0.11993288590604027,
 0.09375838926174497,
 0.19060402684563763,
 0.085906040268456385,
 0.14087248322147652,
 0.19583892617449666,
 0.22724832214765103,
 0.14087248322147652,
 0.075436241610738253,
 0.20107382550335573,
 0.18798657718120809,
 0.19060402684563763,
 0.21677852348993293,
 0.13825503355704699,
 0.17489932885906043,
 0.20369127516778526]

In [47]:
q

[0.27174496644295304,
 0.067583892617449667,
 0.096375838926174504,
 0.29530201342281881,
 0.16704697986577183,
 0.18536912751677856,
 0.21939597315436243,
 0.22201342281879197,
 0.19322147651006713,
 0.10684563758389262,
 0.19060402684563763,
 0.21677852348993293,
 0.19060402684563763,
 0.20892617449664433,
 0.17228187919463089,
 0.15134228187919466,
 0.21677852348993293]

## nn

In [135]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, AveragePooling1D

pred_train = np.zeros((len(y_all), 17))
sc,sc_mean = [],[]

for itr, ite in cv_folds:
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=106))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(17, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
    model.fit(tr_conc.ix[itr].values, y_all.ix[itr].values,
              validation_data=(tr_conc.ix[ite].values, y_all.ix[ite].values),
              batch_size=100, nb_epoch=250, verbose=2)
    pred_train[ite] = model.predict(tr_conc.ix[ite].values)
    loc_sc = fbeta_score(y_all.ix[ite, :], pred_train[ite] > 0.2, beta=2, average='samples')
    print(loc_sc)
    sc.append(loc_sc)
print('XGB: {:.7f} +- {:.7f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 32383 samples, validate on 8096 samples
Epoch 1/250
1s - loss: 0.1713 - acc: 0.9417 - val_loss: 0.0984 - val_acc: 0.9663
Epoch 2/250
1s - loss: 0.1068 - acc: 0.9644 - val_loss: 0.0913 - val_acc: 0.9676
Epoch 3/250
1s - loss: 0.1005 - acc: 0.9655 - val_loss: 0.0892 - val_acc: 0.9676
Epoch 4/250
1s - loss: 0.0977 - acc: 0.9659 - val_loss: 0.0871 - val_acc: 0.9681
Epoch 5/250
1s - loss: 0.0956 - acc: 0.9661 - val_loss: 0.0858 - val_acc: 0.9681
Epoch 6/250
1s - loss: 0.0945 - acc: 0.9663 - val_loss: 0.0848 - val_acc: 0.9684
Epoch 7/250
1s - loss: 0.0936 - acc: 0.9663 - val_loss: 0.0848 - val_acc: 0.9682
Epoch 8/250
1s - loss: 0.0929 - acc: 0.9665 - val_loss: 0.0838 - val_acc: 0.9685
Epoch 9/250
1s - loss: 0.0923 - acc: 0.9665 - val_loss: 0.0836 - val_acc: 0.9685
Epoch 10/250
1s - loss: 0.0917 - acc: 0.9665 - val_loss: 0.0839 - val_acc: 0.9681
Epoch 11/250
1s - loss: 0.0914 - acc: 0.9666 - val_loss: 0.0829 - val_acc: 0.9685
Epoch 12/250
1s - loss: 0.0906 - acc: 0.9668 - val_loss: 0

In [437]:
0.9301398
0.9281668
0.9283114
0.9260916

0.9260916

In [439]:
cv_folds[4]

[array([    0,     1,     2, ..., 40475, 40477, 40478]),
 array([    5,     8,    10, ..., 40467, 40468, 40476])]

In [31]:
pred_train

array([[  9.99929190e-01,   6.28942676e-17,   2.41957451e-07, ...,
          4.46432103e-14,   7.57077373e-07,   4.85096782e-01],
       [  9.99942660e-01,   3.24663233e-05,   7.39261625e-13, ...,
          7.32313236e-03,   1.13315070e-02,   9.99699116e-01],
       [  9.99994755e-01,   1.49332941e-16,   2.31145729e-17, ...,
          4.05182797e-11,   2.48834181e-06,   9.99659657e-01],
       ..., 
       [  7.98553050e-01,   2.66852612e-06,   1.14681531e-09, ...,
          2.55176425e-03,   4.97260273e-01,   9.99344051e-01],
       [  9.06705201e-01,   2.78187304e-08,   1.21807426e-08, ...,
          3.74744704e-04,   3.46397818e-03,   8.91566396e-01],
       [  9.99999881e-01,   4.51274786e-08,   7.94609534e-09, ...,
          6.08505234e-02,   1.83584113e-02,   6.85956271e-04]])

In [32]:
ma = 0
q=[0.2]*17
for t in range(17):
    ib = 0.2
    for i in np.linspace(0.05, 0.3, 50):
        q[t] = i
        cur = fbeta_score(y_all, np.greater(np.array(pred_train), q), beta=2, average='samples')
        if (cur > ma):
            ma = cur
            ib = i
    q[t] = ib
print(ma)

0.928760026482


In [ ]:
0.930672899448
0.929144586029
0.929116884774
0.927450949963

In [33]:
q

[0.22346938775510206,
 0.11632653061224489,
 0.075510204081632656,
 0.16224489795918368,
 0.24897959183673468,
 0.2,
 0.23367346938775507,
 0.18265306122448977,
 0.16734693877551021,
 0.21836734693877552,
 0.21326530612244898,
 0.21326530612244898,
 0.18265306122448977,
 0.22857142857142854,
 0.16224489795918368,
 0.16224489795918368,
 0.17755102040816328]

## голосование

In [172]:
vote = te1[0] * 0
for i in range(4):
    vote = vote + (np.array(te1[i]) > q_all[0]) * 1.0
    
for i in range(4):
    vote = vote + (np.array(te2[i]) > q_all[1]) * 1.0    

for i in range(4):
    vote = vote + (np.array(te3[i]) > q_all[2]) * 1.0 
    
for i in range(5):
    vote = vote + (np.array(te4[i]) > q_all[3]) * 1.0 
    
vote = vote + (np.array(ave5) > 0.2) * 1.0 
vote = vote + (np.array(ave6) > 0.2) * 1.0 



In [173]:
vote

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
1,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
2,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,13.0,19.0,0.0,0.0,0.0,0.0,19.0
4,18.0,0.0,16.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
6,19.0,0.0,0.0,0.0,0.0,13.0,3.0,8.0,0.0,0.0,19.0,19.0,6.0,19.0,0.0,0.0,16.0
7,18.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,12.0,0.0,19.0,0.0,0.0,19.0
8,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
9,19.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12.0,19.0,19.0,0.0,0.0,0.0,16.0


In [135]:
print((vote == 4.0).sum().sum())
print((vote == 3.0).sum().sum())
print((vote == 2.0).sum().sum())
print((vote == 1.0).sum().sum())
print((vote == 0.0).sum().sum())

6232
6844
9594
16282
784888


## Test

In [73]:
clf_multilabel = OneVsRestClassifier(XGBClassifier(max_depth=2,
                                                       learning_rate=0.1,
                                                       n_estimators=100,
                                                       min_child_weight=1))
    #clf_multilabel = OneVsRestClassifier(LogisticRegression())
clf_multilabel.fit(tr_conc.ix[:].values, y_all.ix[:].values)
pr_xgb = clf_multilabel.predict_proba(te_conc.ix[:].values)

In [74]:
xgb_test = pr_xgb

In [142]:
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=106))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(17, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
    model.fit(tr_conc.ix[:].values, y_all.ix[:].values,
              validation_data=(tr_conc.ix[ite].values, y_all.ix[ite].values),
              batch_size=500, nb_epoch=250, verbose=0)
    nn_test2 = model.predict(te_conc.ix[:].values)

/home/fao3864/anaconda3/lib/python3.6/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


In [77]:
clf_multilabel = OneVsRestClassifier(LogisticRegression())

clf_multilabel.fit(tr_conc.ix[:].values, y_all.ix[:].values)
lin_test = clf_multilabel.predict_proba(te_conc.ix[:].values)

In [78]:
clf_multilabel = OneVsRestClassifier(ExtraTreesClassifier(n_estimators=500, criterion='gini', max_depth=None))

clf_multilabel.fit(tr_conc.ix[:].values, y_all.ix[:].values)
ext_test = clf_multilabel.predict_proba(te_conc.ix[:].values)

In [79]:
clf_multilabel = OneVsRestClassifier(lgb.sklearn.LGBMClassifier(n_estimators=150, seed=0, **params)) 

clf_multilabel.fit(tr_conc.ix[:].values, y_all.ix[:].values)
lgb_test = clf_multilabel.predict_proba(te_conc.ix[:].values)

In [143]:
my_test = (lin_test*0.14 + xgb_test*0.3+ ext_test*0.14+lgb_test*0.14+nn_test*0.14+nn_test2*0.14)

In [81]:
my_test =  (lgb_test+ext_test+lin_test+nn_test+xgb_test)/5.0

In [144]:
bl = pd.DataFrame(pd.DataFrame(my_test).values * 0.65 + pd.DataFrame(ave4).values * 0.35)

In [145]:
p_test = np.array(bl) > q

In [297]:
p_test = np.array(ave5) > 0.2

In [234]:
for i in p_test[:]:
    if (i.sum() == 0):
        print(i.sum())

In [237]:
pt.sum()

0     58256
1       681
2      4787
3       553
4       203
5     12841
6     15319
7     14456
8       610
9       208
10    11105
11    23717
12     5503
13     7837
14      251
15     2218
16    45478
dtype: int64

In [252]:
pt.sum().sum()

204023

In [236]:
pt = pd.DataFrame(p_test)

In [250]:
p_test.sum()

203940

In [169]:
for i in pt.index:
    if pt.ix[i,:].sum() == 0:
        print(i)

In [288]:
p_test = model.predict(te_conc.ix[:].values)

In [301]:
clf_multilabel.fit(tr_conc.ix[:].values, y_all.ix[:].values)


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [302]:
p_test = clf_multilabel.predict_proba(te_conc.ix[:].values)


In [174]:
p_test = np.array(vote) > 9

In [303]:
p_test = np.array(ave) > q

In [310]:
q_all[3]

[0.29999999999999999,
 0.13673469387755102,
 0.16734693877551021,
 0.27959183673469384,
 0.12142857142857143,
 0.22857142857142854,
 0.24897959183673468,
 0.23877551020408161,
 0.13163265306122449,
 0.090816326530612251,
 0.21326530612244898,
 0.19795918367346937,
 0.21326530612244898,
 0.19285714285714284,
 0.1010204081632653,
 0.16734693877551021,
 0.24387755102040815]

In [204]:
len(p_test)

61191

In [85]:
len(p_test)

61191

In [194]:
p_test = np.array(blend) > q_all[4]

In [92]:
p_test  = np.array(p_test) > 0.2

In [163]:
label_map

{'agriculture': 11,
 'artisinal_mine': 1,
 'bare_ground': 15,
 'blooming': 3,
 'blow_down': 9,
 'clear': 16,
 'cloudy': 2,
 'conventional_mine': 4,
 'cultivation': 10,
 'habitation': 13,
 'haze': 12,
 'partly_cloudy': 5,
 'primary': 0,
 'road': 6,
 'selective_logging': 8,
 'slash_burn': 14,
 'water': 7}

In [146]:
categories = np.array(['primary', 'artisinal_mine', 'cloudy','blooming','conventional_mine',
                       'partly_cloudy','road','water','selective_logging','blow_down',
                       'cultivation','agriculture','haze','habitation',
                       'slash_burn','bare_ground','clear'])

In [147]:
sample_submission = pd.read_csv('../data/sample_submission_v2.csv')

In [166]:
len(p_test)

61191

In [148]:
res = []
for i in range(len(p_test)):
    res += [' '.join(categories[p_test[i]])]
    if res[i] == ' ':
        res[i] = 'primary'
        print('primary')

In [149]:
sample_submission['tags'] = res

In [150]:
sample_submission[:5]

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary cultivation agriculture clear
4,test_4,primary cloudy partly_cloudy


In [169]:
sample_submission[:5]

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary cultivation agriculture clear
4,test_4,primary cloudy partly_cloudy


In [151]:
sample_submission.to_csv("stack_3.csv", index=None)